In [2]:
import pandas as pd
import requests

import re

from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

In [2]:
# TFT 시즌 8 조합 정보 가져오기
# https://lolchess.gg/synergies/set8

url = 'https://lolchess.gg/synergies/set8'
headers = {
    'user-agent' : 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Mobile Safari/537.36',
    "Accept-Language": "ko-KR"
    
}

headers_en = {
    'user-agent' : 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Mobile Safari/537.36',
    "Accept-Language": "en-US"
}

response = requests.get(url, headers = headers)
response_en = requests.get(url)


if response.status_code == 200:
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
else:
    print("REQUEST ERROR !!")
    
if response_en.status_code == 200:
    html = response_en.text
    soup_en = BeautifulSoup(html, 'html.parser')
else:
    print("REQUEST(ENG) ERROR !!")
    



In [3]:
synergies = soup.select('#wrapper > div.container.mt-5.px-0.px-md-3.guide-synergies > div.mb-5 > div.row.row-normal > div.col-lg-6')

synergies_en = soup_en.select('#wrapper > div.container.mt-5.px-0.px-md-3.guide-synergies > div.mb-5 > div.row.row-normal > div.col-lg-6')

In [4]:
# 시너지 타이틀
synergies[0].select_one('div.d-inline-block.align-middle > span').text
synergies_en[0].select_one('div.d-inline-block.align-middle > span').text

'A.D.M.I.N.'

In [5]:
#챔피언
synergies[0].select('div.guide-synergy__champions > a')

[<a class="d-inline-block" data-toggle="tooltip" data-tooltip-url="https://lolchess.gg/tooltip/champion/set8/Lulu" href="https://lolchess.gg/champions/set8/lulu">
 <div class="guide-synergy__champions-champion cost-1 set8">
 <img alt="룰루" src="//cdn.lolchess.gg/upload/images/champions/Lulu_1668168372-Lulu.jpg" width="48"/>
 <span class="cost">$1</span>
 <span class="name">룰루</span>
 </div>
 </a>,
 <a class="d-inline-block" data-toggle="tooltip" data-tooltip-url="https://lolchess.gg/tooltip/champion/set8/Poppy" href="https://lolchess.gg/champions/set8/poppy">
 <div class="guide-synergy__champions-champion cost-1 set8">
 <img alt="뽀삐" src="//cdn.lolchess.gg/upload/images/champions/Poppy_1668168526-Poppy.jpg" width="48"/>
 <span class="cost">$1</span>
 <span class="name">뽀삐</span>
 </div>
 </a>,
 <a class="d-inline-block" data-toggle="tooltip" data-tooltip-url="https://lolchess.gg/tooltip/champion/set8/Annie" href="https://lolchess.gg/champions/set8/annie">
 <div class="guide-synergy__cha

In [8]:
s = synergies_en[0].select('div.guide-synergy__champions > a')[0].text
re.sub('[^a-zA-Z]*', '' , s)

'Blitzcrank'

In [12]:
s = synergies[0].select('div.guide-synergy__champions > a')[0].text
re.sub('[^가-힣]*', '', s)

'룰루'

In [10]:
#시너지 설명
synergies[0].select_one('div.guide-synergy__description').text

'\n                            라운드마다 기계유망주는 강력한 효과를 지닌 무작위 개조 무기를 제작합니다. 무기는 한 라운드 이후 망가집니다. 기계유망주는 장착한 아이템 수에 비례해 입히는 피해가 증가하고 받는 피해가 감소합니다.\n                        '

In [11]:
#시너지 효과
synergies[0].select('div.guide-synergy__stats > div.d-block')

[<div class="d-block">(3) 아이템 1개 제작, 장착한 아이템당 3%</div>,
 <div class="d-block">(5) 아이템 2개 제작, 장착한 아이템당 12%</div>]

In [188]:
# 시너지 테이블 만들기

synergy_df = pd.DataFrame()

synergy_list = dict()

for synergy in synergies:
    champ = []
    name = synergy.select_one('div.d-inline-block.align-middle > span').text
    
    for char in synergy.select('div.float-right.d-none.d-sm-inline-block.align-middle > div.guide-synergy__champions > a'):
        champ.append(re.sub('[^가-힣]*', '', char.select_one('span.name').text))
    
    description_tag = synergy.select_one('div.guide-synergy__description')
    if description_tag:
        description = re.sub('\n *[^가-힣]', '',description_tag.text)
    else:
        description = ''
    
    effects = ([x.text for x in synergy.select('div.guide-synergy__stats > div.d-block')])
    
    tmp = pd.DataFrame({
        'synergy' : name,
        'champ' : [champ],
        'description' : description,
        'effects' : [effects]
    })
    
    synergy_df = pd.concat([synergy_df, tmp])


In [21]:
# 시너지 테이블 만들기

synergy_en_df = pd.DataFrame()

synergy_list = dict()

for synergy_en in synergies_en:
    champ = []
    name = synergy_en.select_one('div.d-inline-block.align-middle > span').text
    
    for char in synergy_en.select('div.float-right.d-none.d-sm-inline-block.align-middle > div.guide-synergy__champions > a'):
        champ.append(re.sub('[^a-zA-Z]*', '' , char.select_one('span.name').text))
    
    description_tag = synergy_en.select_one('div.guide-synergy__description')
    if description_tag:
        description = re.sub('[^a-zA-Z]*', '', description_tag.text)
    else:
        description = ''
    
    effects = ([x.text for x in synergy_en.select('div.guide-synergy__stats > div.d-block')])
    
    tmp = pd.DataFrame({
        'synergy' : name,
        'champ' : [champ],
        'description' : description,
        'effects' : [effects]
    })
    
    synergy_en_df = pd.concat([synergy_en_df, tmp])



In [24]:
synergy_df.reset_index(drop=True, inplace=True)
synergy_en_df.reset_index(drop=True, inplace=True)

In [25]:
synergy_df.to_csv('./data/TFT_S8_Synergy.csv')
synergy_en_df.to_csv('./data/TFT_S8_Synergy_en.csv')

In [207]:
synergy_df

,synergy,champ,description,effects
0,기계유망주,"[룰루, 뽀삐, 애니, 조이, 누누]",라운드마다 기계유망주는 강력한 효과를 지닌 무작위 개조 무기를 제작합니다. 무기는 ...,"[(3) 아이템 1개 제작, 장착한 아이템당 3%, (5) 아이템 2개 제작, 장착..."
1,동물특공대,"[나서스, 사일러스, 징크스, 리븐, 베인, 미스포츈]",동물특공대는 챔피언을 처치해 명성을 얻습니다. 적 처치를 축하하면서 명성 점수 당 ...,"[(3) 공격력 10% 및 주문력 10, (5) 공격력 35% 및 주문력 35, (..."
2,레이저단,"[레넥톤, 애쉬, 야스오, 세나, 세주아니, 제드, 모데카이저]",레이저단 요원이 공격하거나 공격을 받으면 전투 드론이 요원의 대상에게 마법 피해를 ...,"[(3) 마법 피해량 25., (5) 마법 피해량 45., (7) 마법 피해량 80..."
3,메카: 프라임,"[오공, 드레이븐, 잭스, 세트, 레오나]",메카 선택기를 사용해 프라임을 선택하세요.전투 시작: 프라임이 가장 가까운 메카들과...,"[(3) 프라임의 주문력이 60, 공격력이 60% 증가합니다., (5) 모든 메카의..."
4,민간인,"[갈리오, 시비르, 잔나]",민간인이 생존해 있으면 아군이 민간인을 보호하기 위해 2초마다 마나를 얻습니다.,"[(1) 마나 +2, (2) 마나 +4, (3) 마나 +10]"
5,별 수호자,"[럭스, 렐, 유미, 닐라, 카이사, 에코, 탈리야, 신드라]",모든 마나 획득량이 증가합니다.,"[(3) 마나 40%, (5) 마나 70%, (7) 마나 120%, (9) 마나 2..."
6,병기고,[아펠리오스],NaN,[(1) 병기고 특성을 가진 챔피언을 전장에 배치할 때 장비할 무기를 선택할 수 있...
7,우세,"[갱플랭크, 리신, 말파이트]",NaN,[(3) 전투 시작: 멋진 자세를 취해 아군의 피해량을 18% 증가시킵니다. 이 효...
8,위협,"[람머스, 벨코즈, 초가스, 벨베스, 아우렐리온솔, 자크, 우르곳, 피들스틱]",NaN,"[(1) 위협은 특성 추가 효과가 없지만, 강력한 스킬을 사용하고 기본 능력치를 증..."
9,자동방어체계,"[블리츠크랭크, 카밀, 르블랑, 소라카]",자동방어체계는 매 게임 플레이어에 맞는 설정을 프로그래밍합니다.처치 시 챔피언 (팀...,"[(2) [자동방어체계 원인과 결과 초기 설정], (4) [또 다른 결과 프로그램에..."


In [26]:
synergy_en_df

,synergy,champ,description,effects
0,A.D.M.I.N.,"[Blitzcrank, Camille, LeBlanc, Soraka]",ADMINprogramsacustomconfigurationperplayereach...,"[(2) [Initialize ADMIN cause and effect], (4) ..."
1,Anima Squad,"[Nasus, Sylas, Jinx, Riven, Vayne, MissFortune]",AnimaSquadmembersbuildfameforscoringchampionki...,"[(3) 10% Attack Damage and 10 Ability Power, (..."
2,Arsenal,[Aphelios],,"[(1) When deploying the Arsenal champion, play..."
3,Civilian,"[Galio, Sivir, Janna]",IfthereisaCivilianaliveyourteamisinspiredtopro...,"[(1) +2 Mana, (2) +4 Mana, (3) +10 Mana]"
4,Gadgeteen,"[Lulu, Poppy, Annie, Zoe, Nunu]",EachroundGadgeteenscreaterandommodifiedweaponw...,"[(3) Create 1 item, 3% per item., (5) Create 2..."
5,LaserCorps,"[Ashe, Renekton, Yasuo, Senna, Sejuani, Zed, M...",WhenaLaserCorpagentattacksorishitbyanattackthe...,"[(3) 25 magic damage., (6) 45 magic damage., (..."
6,Mecha: PRIME,"[Wukong, Draven, Jax, Sett, Leona]",UsetheMechaselectoritemtochooseaPRIMECombatsta...,[(3) The PRIME gains 60 Ability Power and 60% ...
7,Ox Force,"[Talon, Annie, Fiora, Alistar, Viego, Aphelios]",OxForceunitsgainarangeofAttackSpeedthatrampsas...,"[(2) 15% - 30% Attack Speed., (4) 40% - 80% At..."
8,Star Guardian,"[Lux, Rell, Yuumi, KaiSa, Nilah, Ekko, Taliyah...",GainmoreManafromallsources,"[(3) 40% additional mana, (5) 70% additional m..."
9,Supers,"[Gangplank, LeeSin, Malphite]",,[(3) Combat start: strike a pose that grants y...


## 챔피언 테이블

In [32]:
synergies[0].select('div.float-right.d-none.d-sm-inline-block.align-middle > div.guide-synergy__champions > a ')[0]

<a class="d-inline-block" data-toggle="tooltip" data-tooltip-url="https://lolchess.gg/tooltip/champion/set8/Lulu" href="https://lolchess.gg/champions/set8/lulu">
<div class="guide-synergy__champions-champion cost-1 set8">
<img alt="룰루" src="//cdn.lolchess.gg/upload/images/champions/Lulu_1668168372-Lulu.jpg" width="48"/>
<span class="cost">$1</span>
<span class="name">룰루</span>
</div>
</a>

In [40]:
champ_cost_df = pd.DataFrame()
for synergy_en in synergies_en:
    for char in synergy_en.select('div.float-right.d-none.d-sm-inline-block.align-middle > div.guide-synergy__champions > a'):
        champ = re.sub('[^a-zA-Z]*', '' , char.select_one('span.name').text)
        cost = char.select_one('span.cost').text.replace("$", '')
        
        tmp = pd.DataFrame({
            'champ' : [champ],
            'cost' : [cost]
        })
        champ_cost_df = pd.concat([champ_dict, tmp])

In [42]:
champ_cost_df.reset_index(drop=True, inplace=True)

In [43]:
champ_cost_df.to_csv('./data/champ_cost.csv')

### 아이템, 증강 이름 크롤링

아이템 : 
https://lolchess.gg/items/set8

증강 :
https://lolchess.gg/guide/augments/set8 </br>
https://lolchess.gg/guide/augments/set8?tier=1</br>
https://lolchess.gg/guide/augments/set8?tier=2</br>
https://lolchess.gg/guide/augments/set8?tier=3

In [1]:
import requests

from bs4 import BeautifulSoup
from collections import defaultdict


items_dict = dict()
champ_augments_dict = defaultdict(list)
augments_dict = dict()

def toNumber(s):
    if 'III' in s:
        return s.replace('III', '3')
    elif 'II' in s:
        return s.replace('II', '2')
    elif 'I' in s:
        return s.replace('I', '1')
    return s

In [ ]:
from ast import literal_eval

# String -> List
def string_to_dict(s):
    return literal_eval(s)

In [3]:
urls = ['https://lolchess.gg/items/set8',
        'https://lolchess.gg/guide/augments/set8', 
        'https://lolchess.gg/guide/augments/set8?tier=1', 
        'https://lolchess.gg/guide/augments/set8?tier=2', 
        'https://lolchess.gg/guide/augments/set8?tier=3']

headers = {
    'user-agent' : 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Mobile Safari/537.36',
    "Accept-Language": "ko-KR"
    
}

headers_en = {
    'user-agent' : 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Mobile Safari/537.36',
    "Accept-Language": "en-US"
}

for url in urls:
    
    if 'item' in url:
        res = requests.get(url, headers=headers)
        res_en = requests.get(url, headers=headers_en)

        s = BeautifulSoup(res.text, "html.parser")
        s_en = BeautifulSoup(res_en.text, 'html.parser')

        items = s.select('#wrapper > div.guide-items > div:nth-child(9) > table > tbody > tr')
        items_en = s_en.select('#wrapper > div.guide-items > div:nth-child(9) > table > tbody > tr')

        for item, item_en in zip(items, items_en):
            name = item.select_one('td.pl-2 > span').text
            name_en = item_en.select_one('td.pl-2 > span').text
            name_en = re.sub("[\'\’ ]", '', name_en)


            items_dict[str.lower(name_en)] = name
        
    elif 'tier' not in url:
        res = requests.get(url, headers=headers)
        res_en = requests.get(url, headers=headers_en)

        s = BeautifulSoup(res.text, "html.parser")
        s_en = BeautifulSoup(res_en.text, 'html.parser')

        augs = s.select('#wrapper > div.guide-chosen > div > div.guide-augments-box > div > div')
        augs_en = s_en.select('#wrapper > div.guide-chosen > div > div.guide-augments-box > div > div')

        for aug, aug_en in zip(augs, augs_en):
            
            champ = re.search('[A-za-z]*(?=.jpg)', aug.select_one('img')['src']).group(0)
            name = toNumber(re.sub('[^가-힣A-Z!\+]', '' ,aug.select_one('h3').text))
            name_en = toNumber(re.sub('[^a-zA-Z!\+]', '' ,aug_en.select_one('h3').text))

            champ_augments_dict[str.lower(champ)] = champ_augments_dict[str.lower(champ)] + [{str.lower(name_en) : name}]
            
    elif 'tier' in url:
        res = requests.get(url, headers=headers)
        res_en = requests.get(url, headers=headers_en)

        s = BeautifulSoup(res.text, "html.parser")
        s_en = BeautifulSoup(res_en.text, 'html.parser')

        augs = s.select('#wrapper > div.guide-chosen > div > div.guide-augments-box > div > div')
        augs_en = s_en.select('#wrapper > div.guide-chosen > div > div.guide-augments-box > div > div')

        for aug, aug_en in zip(augs, augs_en):
            name = toNumber(re.sub('[^가-힣A-Z!\+]', '' ,aug.select_one('h3').text))
            name_en = toNumber(re.sub('[^a-zA-Z!\+]', '' ,aug_en.select_one('h3').text))

            augments_dict[str.lower(name_en)] = name

In [5]:
# 증강 수동 처리
augments_dict['axiomarc1'] = '원칙의 원형낫1'
augments_dict['axiomarc2'] = '원칙의 원형낫2'
augments_dict['maxlevel10'] = '레벨 업!'
augments_dict['highendshopping'] = '품격있는 쇼핑'
augments_dict['heartemblem'] = '선의심장'

In [ ]:
import pickle

In [ ]:
# 사전 pickle 저장

with open('./data/champ_augments_dict.pickle','wb') as f:
    pickle.dump(champ_augments_dict, f)

with open('./data/augments_dict.pickle','wb') as f:
    pickle.dump(augments_dict, f)
    
with open('./data/items_dict.pickle','wb') as f:
    pickle.dump(items_dict, f)

In [8]:
augments_dict

{'battlemage1': '전투마법사1',
 'celestialblessing1': '천상의축복1',
 'cybernetic1mplants1': '사이버네틱이식술1',
 'cyberneticshell1': '사이버네틱외피1',
 'cyberneticuplink1': '사이버네틱통신1',
 'exiles1': '추방자1',
 'standunited1': '단결된의지1',
 'electrocharge1': '고전압1',
 'featherweights1': '경량급1',
 'firstaidkit': '응급처치키트',
 'futuresight1': '예견1',
 '1temgrabbag1': '아이템꾸러미1',
 'ludensecho1': '루덴의메아리1',
 'makeshiftarmor1': '임시변통방어구1',
 'knifesedge1': '나이프의날1',
 'pandoras1tems': '판도라의아이템',
 'recombobulator': '대격변생성기',
 'secondwind1': '재생의바람1',
 'thrillofthehunt1': '사냥의전율1',
 'tinytitans': '꼬마거인',
 'triforce1': '에깃든힘1',
 'afk': '자리비움',
 'bandofthieves1': '도둑무리1',
 'bigfriend1': '커다란친구1',
 'consistency': '일관성',
 'lategamespecialist': '후반전문가',
 'pandorasbench': '판도라의대기석',
 'preparation1': '준비1',
 'adm1nheart': '자동방어체계심장',
 'aegisheart': '방패대심장',
 'animasquadheart': '동물특공대심장',
 'brawlerheart': '싸움꾼심장',
 'spellslingerheart': '주문투척자심장',
 'sureshotheart': '특등사수심장',
 'defenderheart': '엄호대심장',
 'duelistheart': '결투가심장',
 'gadgeteenh